In [ ]:
#READ ME
#This is the original single file handler as written Darryl Taylor
#This script can provide useful information on your data but is not capable of batch processing.


import pandas as pd
import numpy as np

def calculate_stress_strain(file_path, plate_size_mm, rod_length_mm, rod_diameter_mm,
                            theta_r_deg, theta_z_deg, output_csv="stress_strain_results.csv"):
    # Constants
    E_rod = 2e11  # Pa (Young's modulus of stainless steel)
    
    # Load data
    df = pd.read_csv(file_path)

    # Convert plate size to meters and calculate area (m²)
    plate_size_m = plate_size_mm / 1000
    plate_area_m2 = plate_size_m ** 2

    # Convert rod dimensions to meters
    L = rod_length_mm / 1000
    d = rod_diameter_mm / 1000

    # Moment of inertia (m^4)
    I = (np.pi * d**4) / 64

    # Rod stiffness (N/m)
    k_rod = (3 * E_rod * I) / (L**3)

    # Convert angles to radians
    theta_r = np.radians(theta_r_deg)
    theta_z = np.radians(theta_z_deg)

    # Adjust displacement and force
    df["Adjusted Displacement (μm)"] = df["ZTip Displacement(um)"] * np.cos(theta_r)
    df["Adjusted Force (μN)"] = df["ZForce(uN)"] * np.cos(theta_z)

    # Calculate force lost to rod bending (convert μm to m)
    displacement_m = df["Adjusted Displacement (μm)"] * 1e-6
    F_rod_N = k_rod * displacement_m  # in N
    F_rod_uN = F_rod_N * 1e6  # convert to μN

    # Final force applied to sample
    df["Corrected Force (μN)"] = df["Adjusted Force (μN)"] - F_rod_uN

    # Compute stress in μPa
    df["Stress (μPa)"] = df["Corrected Force (μN)"] / plate_area_m2

    # Extract compress cycles
    df["Cycle_Label"] = df["Cycle"].astype(str).str.extract(r"(\d+-Compress)")[0]

    # Calculate strain per cycle
    strain_values = [np.nan] * len(df)
    for label, group in df.groupby("Cycle_Label"):
        if pd.isna(label):
            continue
        initial_disp = group["Adjusted Displacement (μm)"].iloc[0]
        initial_length = group["Current Size (um)"].iloc[0]
        for idx in group.index:
            strain_values[idx] = (df.loc[idx, "Adjusted Displacement (μm)"] - initial_disp) / initial_length

    df["Strain"] = strain_values

    # Save result
    df.to_csv(output_csv, index=False)
    print(f"Stress and strain calculations saved to {output_csv}")
    return df

In [2]:
import matplotlib.pyplot as plt

def plot_stress_strain(df, cycle_label=None, stress_col="Stress (μPa)", strain_col="Strain"):
    """
    Plots stress vs. strain for one or more compression cycles.

    Parameters:
    - df: DataFrame with processed stress-strain data
    - cycle_label: string or list of cycle labels (e.g., "1-Compress"), or None for all
    - stress_col: column name for stress (default = "Stress (μPa)")
    - strain_col: column name for strain (default = "Strain")
    """
    plt.figure(figsize=(8, 6))
    
    if cycle_label is None:
        cycle_groups = df.groupby("Cycle_Label")
    else:
        if isinstance(cycle_label, str):
            cycle_label = [cycle_label]
        cycle_groups = df[df["Cycle_Label"].isin(cycle_label)].groupby("Cycle_Label")

    for label, group in cycle_groups:
        plt.plot(group[strain_col], group[stress_col], label=label)

    plt.xlabel("Strain")
    plt.ylabel(stress_col)
    plt.title("Stress vs. Strain")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [3]:
calculate_stress_strain(
    file_path="multicycle.csv",       # or full path if needed
    plate_size_mm=2,                  # square plate side length
    rod_length_mm=45,                 # rod length in mm
    rod_diameter_mm=0.5588,           # rod diameter in mm
    theta_r_deg=0,                    # rod tilt in r-direction
    theta_z_deg=10,                   # rod tilt in z-direction
    output_csv="stress_strain_corrected.csv"
)

FileNotFoundError: [Errno 2] No such file or directory: 'multicycle.csv'

In [ ]:
# Load your processed data first
import pandas as pd
df = pd.read_csv("stress_strain_corrected.csv")

# Plot all compression cycles
plot_stress_strain(df)

In [ ]:
#Or to plot a specific cycle (e.g., "1-Compress"):

plot_stress_strain(df, cycle_label="1-Compress")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_storage_and_shear_modulus(df, displacement_col="ZTip Displacement(um)", 
                                        force_col="ZForce(uN)", temp_col="Temperature (C)"):
    # Prepare required columns
    df["Adjusted Displacement (μm)"] = df[displacement_col]
    df["Stress (μPa)"] = df[force_col]  # Assume pre-corrected or approximate
    df["Temperature (K)"] = df[temp_col] + 273.15
    df["Cycle_Label"] = df["Cycle"].astype(str).str.extract(r"(\d+-Compress)")[0]

    # If strain is not yet defined, estimate it
    if "Strain" not in df.columns:
        strain_values = [np.nan] * len(df)
        for label, group in df.groupby("Cycle_Label"):
            if pd.isna(label):
                continue
            initial_disp = group["Adjusted Displacement (μm)"].iloc[0]
            initial_length = group["Current Size (um)"].iloc[0]
            for idx in group.index:
                strain_values[idx] = (df.loc[idx, "Adjusted Displacement (μm)"] - initial_disp) / initial_length
        df["Strain"] = strain_values

    # Storage modulus and shear modulus computation
    results = []
    for label, group in df.groupby("Cycle_Label"):
        if pd.isna(label):
            continue

        group = group.sort_values("Adjusted Displacement (μm)")
        d_disp = np.gradient(group["Adjusted Displacement (μm)"])
        loading_group = group[d_disp > 0]

        if len(loading_group) >= 2:
            strain = loading_group["Strain"]
            stress = loading_group["Stress (μPa)"]
            coeffs = np.polyfit(strain, stress, 1)
            E_prime_uPa = coeffs[0]  # Storage modulus

            T_mean = loading_group["Temperature (K)"].mean()
            G_prime_uPa = E_prime_uPa / 3

            results.append({
                "Cycle_Label": label,
                "Temperature (K)": T_mean,
                "Storage Modulus (μPa)": E_prime_uPa,
                "Shear Modulus (μPa)": G_prime_uPa
            })

    result_df = pd.DataFrame(results)
    return result_df

def plot_moduli(result_df):
    plt.figure(figsize=(10, 6))
    x = result_df["Cycle_Label"]
    
    plt.plot(x, result_df["Storage Modulus (μPa)"], label="Storage Modulus (μPa)", marker='o')
    plt.plot(x, result_df["Shear Modulus (μPa)"], label="Shear Modulus (μPa)", marker='s')
    
    plt.xlabel("Cycle")
    plt.ylabel("Modulus (μPa)")
    plt.title("Storage and Shear Modulus per Cycle")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
df = pd.read_csv("multicycle.csv")
moduli_df = calculate_storage_and_shear_modulus(df)
plot_moduli(moduli_df)